# Helmet Mapping
Source: https://www.kaggle.com/its7171/nfl-baseline-simple-helmet-mapping

In [1]:
# %cd /kaggle
# from IPython.display import clear_output
# pat = input('Personal Access Token:')
# clear_output()
# ! git clone https://$pat@github.com/qAp/nfl_helmet_assignment_kaggle.git

In [2]:
# %cd /kaggle/nfl_helmet_assignment_kaggle/
# ! git pull

In [3]:
# If no Github PAT nor internet
! cp -r /kaggle/input/nfl-helmet-assignment-kaggle/nfl_helmet_assignment_kaggle /kaggle/.

In [4]:
import os, sys
import importlib
import random
from multiprocessing import Pool
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

sys.path.append('/kaggle/nfl_helmet_assignment_kaggle/')
from helmet_tracker.utils.features import add_track_features
from helmet_tracker.models.helmet_mapping import *

In [5]:
# import helmet_tracker
# importlib.reload(helmet_tracker.models.helmet_mapping)

# Setting and loading data

In [6]:
debug = True
CONF_THRE = 0.3
BASE_DIR = '/kaggle/input/nfl-health-and-safety-helmet-assignment/'

labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
if debug:
    tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
    helmets = pd.read_csv(f'{BASE_DIR}/train_baseline_helmets.csv')
else:
    tracking = pd.read_csv(f'{BASE_DIR}/test_player_tracking.csv')
    helmets = pd.read_csv(f'{BASE_DIR}/test_baseline_helmets.csv')

# Adding estimated video frame to tracking

In [7]:
%%time
tracking = add_track_features(tracking)

CPU times: user 1.69 s, sys: 104 ms, total: 1.79 s
Wall time: 1.81 s


In [8]:
if debug:
    sample_keys = random.sample(list(tracking['gameKey'].unique()), 3)
    helmets['gameKey'] = helmets['video_frame'].str.split('_').str[0]
    tracking = tracking[tracking['gameKey'].isin(sample_keys)]
    helmets = helmets[helmets['gameKey'].astype(int).isin(sample_keys)]
    labels = labels[labels['gameKey'].astype(int).isin(sample_keys)]

# Mapping helmet box and tracking data

In [9]:
def _mapping_df(args):
    video_frame, df, tracking, conf_thre = args
    return mapping_df(video_frame, df, tracking, conf_thre=conf_thre)

df_list = list(helmets.groupby('video_frame'))
df_list = [(video_frame, df, tracking, CONF_THRE) for video_frame, df in df_list]

p = Pool(processes=os.cpu_count())
submission_df_list = []
with tqdm(total=len(df_list)) as pbar:
    for this_df in p.imap(_mapping_df, df_list):
        submission_df_list.append(this_df)
        pbar.update(1)
p.close()

  0%|          | 0/2596 [00:00<?, ?it/s]

In [10]:
submission_df = pd.concat(submission_df_list)

p = '/kaggle/working/' + ('submission_helmtrack_debug.csv' if debug else
                          'submission_helmtrack.csv')
submission_df.to_csv(p, index=False)